In [247]:
#importing libraries
import ssl
import requests
from bs4 import BeautifulSoup
import difflib
import csv
import pandas as pd 
import sqlite3
#Preparing context for handeling SSL certificate errors
ctx=ssl.create_default_context()
ctx.check_hostname=False
ctx.verify_mode=ssl.CERT_NONE

In [248]:
#Importing datasets from csv file
genre_set= pd.read_csv('movies.csv')
rating_set=pd.read_csv('ratings.csv')

In [249]:
genre_set.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [250]:
#Dividing movie name and year
value=genre_set.iloc[0].title
print(value)
name,year=value.split('(')
year=year[:-1]
print(name,year)
print(len(genre_set))

Toy Story (1995)
Toy Story  1995
9742


In [251]:
name=genre_set['title'].str.split('(')
genre_set['name']=[value[0][:-1] for value in name]
print(genre_set['name'])

0                                Toy Story
1                                  Jumanji
2                         Grumpier Old Men
3                        Waiting to Exhale
4              Father of the Bride Part II
                       ...                
9737    Black Butler: Book of the Atlantic
9738                 No Game No Life: Zero
9739                                 Flint
9740          Bungo Stray Dogs: Dead Apple
9741          Andrew Dice Clay: Dice Rules
Name: name, Length: 9742, dtype: object


In [252]:
value=genre_set.iloc[29]['title'].split('(')
print(value)

['Shanghai Triad ', 'Yao a yao yao dao waipo qiao) ', '1995)']


In [253]:
temp_val=list()
for temp in name:
    #print(temp[1]) #There are not only single string name but also multiple strings
    temp_val.append(temp[-1][:-1])
#print(temp_val)
genre_set['year']=temp_val

In [254]:
genre_set.head()

,movieId,title,genres,name,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995


In [255]:
#Now work with ratings csv file
ratings_set=pd.read_csv('ratings.csv')
ratings_set.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [256]:
#Perform join operation with genre_set for movie name
ratings_genre=genre_set[['movieId','name','year','genres']].merge(ratings_set,on='movieId',how='inner')

In [257]:
ratings_genre

,movieId,name,year,genres,userId,rating,timestamp
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic,2017,Action|Animation|Comedy|Fantasy,184,4.0,1537109082
100832,193583,No Game No Life: Zero,2017,Animation|Comedy|Fantasy,184,3.5,1537109545
100833,193585,Flint,2017,Drama,184,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple,2018,Action|Animation,184,3.5,1537110021


In [258]:
final_average=ratings_genre.groupby(['movieId','name','year']).aggregate({'userId':'count','rating':'mean'})

In [259]:
final_average

,,,userId,rating
movieId,name,year,,
1,Toy Story,1995,215,3.920930
2,Jumanji,1995,110,3.431818
3,Grumpier Old Men,1995,52,3.259615
4,Waiting to Exhale,1995,7,2.357143
5,Father of the Bride Part II,1995,49,3.071429
...,...,...,...,...
193581,Black Butler: Book of the Atlantic,2017,1,4.000000
193583,No Game No Life: Zero,2017,1,3.500000
193585,Flint,2017,1,3.500000


In [260]:
final_average_use=final_average.merge(genre_set[['movieId','name','year','genres']],on='movieId',how='inner')

In [261]:
final_average_use.head()

,movieId,userId,rating,name,year,genres
0,1,215,3.920930,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,2,110,3.431818,Jumanji,1995,Adventure|Children|Fantasy
2,3,52,3.259615,Grumpier Old Men,1995,Comedy|Romance
3,4,7,2.357143,Waiting to Exhale,1995,Comedy|Drama|Romance
4,5,49,3.071429,Father of the Bride Part II,1995,Comedy


In [262]:
name='Father of the Bride Part II' 
year='1995'
rating=final_average_use[final_average_use.name==name].values[0][2]#rating index 2
m_name=final_average_use[final_average_use.name==name].values[0][3]#movie name index 3
genre=final_average_use[final_average_use.name==name].values[0][5]#movie genres index 5
print(counter,m_name,genre)

3.0714285714285716 Father of the Bride Part II Comedy


In [263]:
#Creating database connection and cursor for reading
connection=sqlite3.connect('movie_database.db')
prev_dataset=pd.read_sql('select * from movies',connection)
connection.close()
prev_dataset.head(10)


,index,Name,Directed by,Produced by,Screenplay by,Based on,Starring,Music by,Cinematography,Edited by,Productioncompanies,Distributed by,Release date,Running time,Country,Language,Budget,Box office
0,0,Nomadland,Chloé Zhao,"Frances Mc, Dormand, Peter Spears, Mollye Ashe...",Chloé Zhao,Nomadland: Surviving America in the Twenty-Fir...,"Frances Mc, Dormand, David Strathairn, Linda M...",Ludovico Einaudi,Joshua James Richards,Chloé Zhao,"Highwayman Films, Hear/Say Productions, Cor Co...",Searchlight Pictures,"September 11, 2020 (2020-09-11) (Venice), Febr...",108 minutes,United States,English,$5 million,$20.7 million
1,1,The Father,Florian Zeller,"David Parfitt, Jean-Louis Livi, Philippe Carca...","Florian Zeller, Christopher Hampton",Le Pèreby Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss, I...",Ludovico Einaudi,Ben Smithard,Yorgos Lamprinos,"F comme Film, Trademark Films, Cine@, AG Studi...","Lionsgate(United Kingdom), UGC Distribution(Fr...","27 January 2020 (2020-01-27) (Sundance), 26 Ma...",97 minutes,None,English,$6 million,$14.1 million
2,2,Judas and the Black Messiah,Shaka King,"Charles D. King, Ryan Coogler, Shaka King","Will Berson, Shaka King",None,"Daniel Kaluuya, Lakeith Stanfield, Jesse Plemo...","Mark Isham, Craig Harris",Sean Bobbitt,Kristan Sprague,"MACRO, Participant, Bron Creative, Proximity",Warner Bros. Pictures,"February 12, 2021",126 minutes,United States,English,$26 million,$6.5 million
3,3,Minari,Lee Isaac Chung,"Dede Gardner, Jeremy Kleiner, Christina Oh",None,None,"Steven Yeun, Han Ye-ri, Alan Kim, Noel Kate Ch...",Emile Mosseri,Lachlan Milne,Harry Yoon,Plan B Entertainment,A24,"February 12, 2021",115 minutes,United States,Korean,$2 million,$15 million
4,4,Mank,David Fincher,"Ceán Chaffin, Eric Roth, Douglas Urbanski",Jack Fincher,None,"Gary Oldman, Amanda Seyfried, Lily Collins, Ar...","Trent Reznor, Atticus Ross",Erik Messerschmidt,Kirk Baxter,"Netflix International Pictures, Flying Studio",Netflix,"November 13, 2020 (2020-11-13) (United States)",131 minutes,United States,English,$25 million,"$122,252"
5,5,Sound of Metal,Darius Marder,"Bill Benz, Kathy Benz, Bert Hamelinck, Sacha B...","Darius Marder, Abraham Marder",None,"Riz Ahmed, Olivia Cooke, Paul Raci, Lauren Rid...","Abraham Marder, Nicolas Becker",Daniël Bouquet,Mikkel E. G. Nielsen,"Caviar, Ward Four, Flat 7 Productions",Amazon Studios,"September 6, 2019 (2019-09-06) (TIFF), Novembe...",120 minutes,United States,"American Sign Language, English",$5.4 million,"$176,177"
6,6,Ma Rainey's Black Bottom,George C. Wolfe,"Denzel Washington, Todd Black, Dany Wolf",Ruben Santiago-Hudson,Ma Rainey's Black Bottomby August Wilson,"Viola Davis, Chadwick Boseman, Glynn Turman, C...",Branford Marsalis,Tobias A. Schliessler,Andrew Mondshein,"Escape Artists, Mundy Lane Entertainment",Netflix,"November 25, 2020 (2020-11-25) (United States)",94 minutes,United States,English,$20–22.5 million,None
7,7,Promising Young Woman,Emerald Fennell,"Margot Robbie, Josey Mc, Namara, Tom Ackerley,...",None,None,"Carey Mulligan, Bo Burnham, Alison Brie, Clanc...",Anthony Willis,Benjamin Kračun,Frédéric Thoraval,"Film, Nation Entertainment, Lucky, Chap Entert...",Focus Features,"January 25, 2020 (2020-01-25) (Sundance), Dece...",113 minutes,None,English,$5–10 million,$16.3 million
8,8,Tenet,Christopher Nolan,"Emma Thomas, Christopher Nolan",None,None,"John David Washington, Robert Pattinson, Eliza...",Ludwig Göransson,Hoyte van Hoytema,Jennifer Lame,"Warner Bros. Pictures, Syncopy",Warner Bros. Pictures,"August 26, 2020 (2020-08-26) (United Kingdom),...",150 minutes,None,English,$200 million,$363.7 million
9,9,Soul,Pete Docter,Dana Murray,None,None,"Jamie Foxx, Tina Fey, Graham Norton, Rachel Ho...","Trent Reznor, Atticus Ross, Jon Batiste","Matt Aspbury, Ian Megibben",Kevin Nolting,"Walt Disney Pictures, Pixar Animation Studios",Walt Disney Studios Motion Pictures,"October 11, 2020 (2020-10-11) (BFI Fest), Dece...",100 minutes,United States,English,$150 million+,

In [264]:
count=0
#'''
for row in final_average_use.itertuples():
    #print(row.name)
#'''
#name='Toy Story'
    for scrap in prev_dataset.itertuples():
        if scrap.Name==row.name:
            print(scrap.Name,row.name)
            prev_dataset.loc[scrap.index,'Average Rating']=row.rating
            prev_dataset.loc[scrap.index,'User Count']=row.userId

Toy Story Toy Story
Sabrina Sabrina
Sense and Sensibility Sense and Sensibility
Leaving Las Vegas Leaving Las Vegas
Babe Babe
Dead Man Walking Dead Man Walking
Restoration Restoration
Pocahontas Pocahontas
Mighty Aphrodite Mighty Aphrodite
Antonia's Line Antonia's Line
Braveheart Braveheart
Anne Frank Remembered Anne Frank Remembered
Apollo 13 Apollo 13
Burnt by the Sun Burnt by the Sun
Ed Wood Ed Wood
Little Women Little Women
Little Women Little Women
Little Women Little Women
Legends of the Fall Legends of the Fall
Miracle on 34th Street Miracle on 34th Street
Pulp Fiction Pulp Fiction
Forrest Gump Forrest Gump
Speed Speed
Blue Sky Blue Sky
Jurassic Park Jurassic Park
Mrs. Doubtfire Mrs. Doubtfire
Philadelphia Philadelphia
Schindler's List Schindler's List
Dear Diary Dear Diary
Ghost Ghost
Aladdin Aladdin
Terminator 2: Judgment Day Terminator 2: Judgment Day
Dances with Wolves Dances with Wolves
Batman Batman
Snow White and the Seven Dwarfs Snow White and the Seven Dwarfs
Beauty and

In [265]:
prev_dataset.head(10)

,index,Name,Directed by,Produced by,Screenplay by,Based on,Starring,Music by,Cinematography,Edited by,Productioncompanies,Distributed by,Release date,Running time,Country,Language,Budget,Box office,Average Rating,User Count
0,0,Nomadland,Chloé Zhao,"Frances Mc, Dormand, Peter Spears, Mollye Ashe...",Chloé Zhao,Nomadland: Surviving America in the Twenty-Fir...,"Frances Mc, Dormand, David Strathairn, Linda M...",Ludovico Einaudi,Joshua James Richards,Chloé Zhao,"Highwayman Films, Hear/Say Productions, Cor Co...",Searchlight Pictures,"September 11, 2020 (2020-09-11) (Venice), Febr...",108 minutes,United States,English,$5 million,$20.7 million,NaN,NaN
1,1,The Father,Florian Zeller,"David Parfitt, Jean-Louis Livi, Philippe Carca...","Florian Zeller, Christopher Hampton",Le Pèreby Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss, I...",Ludovico Einaudi,Ben Smithard,Yorgos Lamprinos,"F comme Film, Trademark Films, Cine@, AG Studi...","Lionsgate(United Kingdom), UGC Distribution(Fr...","27 January 2020 (2020-01-27) (Sundance), 26 Ma...",97 minutes,None,English,$6 million,$14.1 million,NaN,NaN
2,2,Judas and the Black Messiah,Shaka King,"Charles D. King, Ryan Coogler, Shaka King","Will Berson, Shaka King",None,"Daniel Kaluuya, Lakeith Stanfield, Jesse Plemo...","Mark Isham, Craig Harris",Sean Bobbitt,Kristan Sprague,"MACRO, Participant, Bron Creative, Proximity",Warner Bros. Pictures,"February 12, 2021",126 minutes,United States,English,$26 million,$6.5 million,NaN,NaN
3,3,Minari,Lee Isaac Chung,"Dede Gardner, Jeremy Kleiner, Christina Oh",None,None,"Steven Yeun, Han Ye-ri, Alan Kim, Noel Kate Ch...",Emile Mosseri,Lachlan Milne,Harry Yoon,Plan B Entertainment,A24,"February 12, 2021",115 minutes,United States,Korean,$2 million,$15 million,NaN,NaN
4,4,Mank,David Fincher,"Ceán Chaffin, Eric Roth, Douglas Urbanski",Jack Fincher,None,"Gary Oldman, Amanda Seyfried, Lily Collins, Ar...","Trent Reznor, Atticus Ross",Erik Messerschmidt,Kirk Baxter,"Netflix International Pictures, Flying Studio",Netflix,"November 13, 2020 (2020-11-13) (United States)",131 minutes,United States,English,$25 million,"$122,252",NaN,NaN
5,5,Sound of Metal,Darius Marder,"Bill Benz, Kathy Benz, Bert Hamelinck, Sacha B...","Darius Marder, Abraham Marder",None,"Riz Ahmed, Olivia Cooke, Paul Raci, Lauren Rid...","Abraham Marder, Nicolas Becker",Daniël Bouquet,Mikkel E. G. Nielsen,"Caviar, Ward Four, Flat 7 Productions",Amazon Studios,"September 6, 2019 (2019-09-06) (TIFF), Novembe...",120 minutes,United States,"American Sign Language, English",$5.4 million,"$176,177",NaN,NaN
6,6,Ma Rainey's Black Bottom,George C. Wolfe,"Denzel Washington, Todd Black, Dany Wolf",Ruben Santiago-Hudson,Ma Rainey's Black Bottomby August Wilson,"Viola Davis, Chadwick Boseman, Glynn Turman, C...",Branford Marsalis,Tobias A. Schliessler,Andrew Mondshein,"Escape Artists, Mundy Lane Entertainment",Netflix,"November 25, 2020 (2020-11-25) (United States)",94 minutes,United States,English,$20–22.5 million,None,NaN,NaN
7,7,Promising Young Woman,Emerald Fennell,"Margot Robbie, Josey Mc, Namara, Tom Ackerley,...",None,None,"Carey Mulligan, Bo Burnham, Alison Brie, Clanc...",Anthony Willis,Benjamin Kračun,Frédéric Thoraval,"Film, Nation Entertainment, Lucky, Chap Entert...",Focus Features,"January 25, 2020 (2020-01-25) (Sundance), Dece...",113 minutes,None,English,$5–10 million,$16.3 million,NaN,NaN
8,8,Tenet,Christopher Nolan,"Emma Thomas, Christopher Nolan",None,None,"John David Washington, Robert Pattinson, Eliza...",Ludwig Göransson,Hoyte van Hoytema,Jennifer Lame,"Warner Bros. Pictures, Syncopy",Warner Bros. Pictures,"August 26, 2020 (2020-08-26) (United Kingdom),...",150 minutes,None,English,$200 million,$363.7 million,NaN,NaN
9,9,Soul,Pete Docter,Dana Murray,None,None,"Jamie Foxx, Tina Fey, Graham Norton, Rachel Ho...","Trent Reznor, Atticus Ross, Jon Batiste","Matt Aspbury, Ian Megibben",Kevin Nolting,"Walt Disney Pictures, Pixar Animation Studios",Walt Disney Studios Motion Pictures,"O

In [266]:
prev_dataset=prev_dataset.fillna('Not Available')

In [267]:
prev_dataset

,index,Name,Directed by,Produced by,Screenplay by,Based on,Starring,Music by,Cinematography,Edited by,Productioncompanies,Distributed by,Release date,Running time,Country,Language,Budget,Box office,Average Rating,User Count
0,0,Nomadland,Chloé Zhao,"Frances Mc, Dormand, Peter Spears, Mollye Ashe...",Chloé Zhao,Nomadland: Surviving America in the Twenty-Fir...,"Frances Mc, Dormand, David Strathairn, Linda M...",Ludovico Einaudi,Joshua James Richards,Chloé Zhao,"Highwayman Films, Hear/Say Productions, Cor Co...",Searchlight Pictures,"September 11, 2020 (2020-09-11) (Venice), Febr...",108 minutes,United States,English,$5 million,$20.7 million,Not Available,Not Available
1,1,The Father,Florian Zeller,"David Parfitt, Jean-Louis Livi, Philippe Carca...","Florian Zeller, Christopher Hampton",Le Pèreby Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss, I...",Ludovico Einaudi,Ben Smithard,Yorgos Lamprinos,"F comme Film, Trademark Films, Cine@, AG Studi...","Lionsgate(United Kingdom), UGC Distribution(Fr...","27 January 2020 (2020-01-27) (Sundance), 26 Ma...",97 minutes,Not Available,English,$6 million,$14.1 million,Not Available,Not Available
2,2,Judas and the Black Messiah,Shaka King,"Charles D. King, Ryan Coogler, Shaka King","Will Berson, Shaka King",Not Available,"Daniel Kaluuya, Lakeith Stanfield, Jesse Plemo...","Mark Isham, Craig Harris",Sean Bobbitt,Kristan Sprague,"MACRO, Participant, Bron Creative, Proximity",Warner Bros. Pictures,"February 12, 2021",126 minutes,United States,English,$26 million,$6.5 million,Not Available,Not Available
3,3,Minari,Lee Isaac Chung,"Dede Gardner, Jeremy Kleiner, Christina Oh",Not Available,Not Available,"Steven Yeun, Han Ye-ri, Alan Kim, Noel Kate Ch...",Emile Mosseri,Lachlan Milne,Harry Yoon,Plan B Entertainment,A24,"February 12, 2021",115 minutes,United States,Korean,$2 million,$15 million,Not Available,Not Available
4,4,Mank,David Fincher,"Ceán Chaffin, Eric Roth, Douglas Urbanski",Jack Fincher,Not Available,"Gary Oldman, Amanda Seyfried, Lily Collins, Ar...","Trent Reznor, Atticus Ross",Erik Messerschmidt,Kirk Baxter,"Netflix International Pictures, Flying Studio",Netflix,"November 13, 2020 (2020-11-13) (United States)",131 minutes,United States,English,$25 million,"$122,252",Not Available,Not Available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,1269,"Toot, Whistle, Plunk and Boom","Ward Kimball, Charles A. Nichols",Walt Disney,Not Available,Not Available,Bill Thompson Thurl Ravenscroft Loulie Jean N...,Joseph Dubin Sonny Burke (songs) Jack Elliot (...,Not Available,Not Available,Walt Disney Productions,Buena Vista Distribution,"November 10, 1953",10 mins (one reel),Not Available,English,Not Available,Not Available,Not Available,Not Available
1270,1270,Topkapi,Jules Dassin,Jules Dassin,Not Available,The Light of Day1962 novelby Eric Ambler,"Melina Mercouri, Peter Ustinov, Maximilian Sch...",Manos Hadjidakis,Henri Alekan,Roger Dwyre,Filmways,United Artists,"September 2, 1964 (1964-09-02)",120 min,United States,English,Not Available,"$7,000,000",2.5,1.0
1271,1271,The Tortoise and the Hare,Wilfred Jackson,Walt Disney,Not Available,Not Available,"Pinto Colvig, Eddie Holden, Marcellite Garner","Darrell Calker, Frank Churchill",Not Available,Not Available,Walt Disney Productions,United Artists,"January 5, 1935 (1935-01-05)",8 minutes 17 seconds,Not Available,English,Not Available,Not Available,Not Available,Not Available
1272,1272,Torture Money,Harold S. Bucquet,Jack Chertok,Not Available,Not Available,"Edwin Maxwell, George Lynn",Not Available,Not Available,Not Available,Not Available,MGM,"January 2, 1937 (1937-01-02)",21 minutes,United States,English,Not Available,Not Available,Not Available,Not Available


In [268]:
#For checking adding ratings genres for
count=0
#'''
for row1 in genre_set.itertuples():
    #print(row.name)
#'''
#name='Toy Story'
    for scrap1 in prev_dataset.itertuples():
        if scrap1.Name==row1.name:
            print(scrap1.Name,row1.name)
            prev_dataset.loc[scrap1.index,'Genres']=row1.genres

Toy Story Toy Story
Sabrina Sabrina
Sense and Sensibility Sense and Sensibility
Leaving Las Vegas Leaving Las Vegas
Babe Babe
Dead Man Walking Dead Man Walking
Restoration Restoration
Pocahontas Pocahontas
Mighty Aphrodite Mighty Aphrodite
Antonia's Line Antonia's Line
Braveheart Braveheart
Anne Frank Remembered Anne Frank Remembered
Apollo 13 Apollo 13
Burnt by the Sun Burnt by the Sun
Ed Wood Ed Wood
Little Women Little Women
Little Women Little Women
Little Women Little Women
Legends of the Fall Legends of the Fall
Miracle on 34th Street Miracle on 34th Street
Pulp Fiction Pulp Fiction
Forrest Gump Forrest Gump
Speed Speed
Blue Sky Blue Sky
Jurassic Park Jurassic Park
Mrs. Doubtfire Mrs. Doubtfire
Philadelphia Philadelphia
Schindler's List Schindler's List
Dear Diary Dear Diary
Ghost Ghost
Aladdin Aladdin
Terminator 2: Judgment Day Terminator 2: Judgment Day
Dances with Wolves Dances with Wolves
Batman Batman
Snow White and the Seven Dwarfs Snow White and the Seven Dwarfs
Beauty and

In [269]:
prev_dataset.head()

,index,Name,Directed by,Produced by,Screenplay by,Based on,Starring,Music by,Cinematography,Edited by,...,Distributed by,Release date,Running time,Country,Language,Budget,Box office,Average Rating,User Count,Genres
0,0,Nomadland,Chloé Zhao,"Frances Mc, Dormand, Peter Spears, Mollye Ashe...",Chloé Zhao,Nomadland: Surviving America in the Twenty-Fir...,"Frances Mc, Dormand, David Strathairn, Linda M...",Ludovico Einaudi,Joshua James Richards,Chloé Zhao,...,Searchlight Pictures,"September 11, 2020 (2020-09-11) (Venice), Febr...",108 minutes,United States,English,$5 million,$20.7 million,Not Available,Not Available,NaN
1,1,The Father,Florian Zeller,"David Parfitt, Jean-Louis Livi, Philippe Carca...","Florian Zeller, Christopher Hampton",Le Pèreby Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss, I...",Ludovico Einaudi,Ben Smithard,Yorgos Lamprinos,...,"Lionsgate(United Kingdom), UGC Distribution(Fr...","27 January 2020 (2020-01-27) (Sundance), 26 Ma...",97 minutes,Not Available,English,$6 million,$14.1 million,Not Available,Not Available,NaN
2,2,Judas and the Black Messiah,Shaka King,"Charles D. King, Ryan Coogler, Shaka King","Will Berson, Shaka King",Not Available,"Daniel Kaluuya, Lakeith Stanfield, Jesse Plemo...","Mark Isham, Craig Harris",Sean Bobbitt,Kristan Sprague,...,Warner Bros. Pictures,"February 12, 2021",126 minutes,United States,English,$26 million,$6.5 million,Not Available,Not Available,NaN
3,3,Minari,Lee Isaac Chung,"Dede Gardner, Jeremy Kleiner, Christina Oh",Not Available,Not Available,"Steven Yeun, Han Ye-ri, Alan Kim, Noel Kate Ch...",Emile Mosseri,Lachlan Milne,Harry Yoon,...,A24,"February 12, 2021",115 minutes,United States,Korean,$2 million,$15 million,Not Available,Not Available,NaN
4,4,Mank,David Fincher,"Ceán Chaffin, Eric Roth, Douglas Urbanski",Jack Fincher,Not Available,"Gary Oldman, Amanda Seyfried, Lily Collins, Ar...","Trent Reznor, Atticus Ross",Erik Messerschmidt,Kirk Baxter,...,Netflix,"November 13, 2020 (2020-11-13) (United States)",131 minutes,United States,English,$25 million,"$122,252",Not Available,Not Available,NaN


In [270]:
new_dataset=prev_dataset.fillna('Not Available')

In [271]:
new_dataset

,index,Name,Directed by,Produced by,Screenplay by,Based on,Starring,Music by,Cinematography,Edited by,...,Distributed by,Release date,Running time,Country,Language,Budget,Box office,Average Rating,User Count,Genres
0,0,Nomadland,Chloé Zhao,"Frances Mc, Dormand, Peter Spears, Mollye Ashe...",Chloé Zhao,Nomadland: Surviving America in the Twenty-Fir...,"Frances Mc, Dormand, David Strathairn, Linda M...",Ludovico Einaudi,Joshua James Richards,Chloé Zhao,...,Searchlight Pictures,"September 11, 2020 (2020-09-11) (Venice), Febr...",108 minutes,United States,English,$5 million,$20.7 million,Not Available,Not Available,Not Available
1,1,The Father,Florian Zeller,"David Parfitt, Jean-Louis Livi, Philippe Carca...","Florian Zeller, Christopher Hampton",Le Pèreby Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss, I...",Ludovico Einaudi,Ben Smithard,Yorgos Lamprinos,...,"Lionsgate(United Kingdom), UGC Distribution(Fr...","27 January 2020 (2020-01-27) (Sundance), 26 Ma...",97 minutes,Not Available,English,$6 million,$14.1 million,Not Available,Not Available,Not Available
2,2,Judas and the Black Messiah,Shaka King,"Charles D. King, Ryan Coogler, Shaka King","Will Berson, Shaka King",Not Available,"Daniel Kaluuya, Lakeith Stanfield, Jesse Plemo...","Mark Isham, Craig Harris",Sean Bobbitt,Kristan Sprague,...,Warner Bros. Pictures,"February 12, 2021",126 minutes,United States,English,$26 million,$6.5 million,Not Available,Not Available,Not Available
3,3,Minari,Lee Isaac Chung,"Dede Gardner, Jeremy Kleiner, Christina Oh",Not Available,Not Available,"Steven Yeun, Han Ye-ri, Alan Kim, Noel Kate Ch...",Emile Mosseri,Lachlan Milne,Harry Yoon,...,A24,"February 12, 2021",115 minutes,United States,Korean,$2 million,$15 million,Not Available,Not Available,Not Available
4,4,Mank,David Fincher,"Ceán Chaffin, Eric Roth, Douglas Urbanski",Jack Fincher,Not Available,"Gary Oldman, Amanda Seyfried, Lily Collins, Ar...","Trent Reznor, Atticus Ross",Erik Messerschmidt,Kirk Baxter,...,Netflix,"November 13, 2020 (2020-11-13) (United States)",131 minutes,United States,English,$25 million,"$122,252",Not Available,Not Available,Not Available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,1269,"Toot, Whistle, Plunk and Boom","Ward Kimball, Charles A. Nichols",Walt Disney,Not Available,Not Available,Bill Thompson Thurl Ravenscroft Loulie Jean N...,Joseph Dubin Sonny Burke (songs) Jack Elliot (...,Not Available,Not Available,...,Buena Vista Distribution,"November 10, 1953",10 mins (one reel),Not Available,English,Not Available,Not Available,Not Available,Not Available,Not Available
1270,1270,Topkapi,Jules Dassin,Jules Dassin,Not Available,The Light of Day1962 novelby Eric Ambler,"Melina Mercouri, Peter Ustinov, Maximilian Sch...",Manos Hadjidakis,Henri Alekan,Roger Dwyre,...,United Artists,"September 2, 1964 (1964-09-02)",120 min,United States,English,Not Available,"$7,000,000",2.5,1.0,Adventure|Comedy|Thriller
1271,1271,The Tortoise and the Hare,Wilfred Jackson,Walt Disney,Not Available,Not Available,"Pinto Colvig, Eddie Holden, Marcellite Garner","Darrell Calker, Frank Churchill",Not Available,Not Available,...,United Artists,"January 5, 1935 (1935-01-05)",8 minutes 17 seconds,Not Available,English,Not Available,Not Available,Not Available,Not Available,Not Available
1272,1272,Torture Money,Harold S. Bucquet,Jack Chertok,Not Available,Not Available,"Edwin Maxwell, George Lynn",Not Available,Not Available,Not Available,...,MGM,"January 2, 1937 (1937-01-02)",21 minutes,United States,English,Not Available,Not Available,Not Available,Not Available,Not Available


In [274]:
new_connection=sqlite3.connect('wikiapi/api_database.sqlite3')
new_dataset.to_sql('movies',new_connection)
new_connection.close()


C:\Users\Soumik\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(
